In [17]:
import math
from typing import List, Optional, Tuple, Union

import torch  # PyTorch를 임포트하여 딥러닝 연산을 수행합니다.
import torch.nn.functional as F
import torch.utils.checkpoint
from torch import nn  # 신경망 모듈을 임포트합니다.

from transformers.activations import ACT2FN  # 다양한 활성화 함수들을 함수 이름과 연결해주는 매핑 테이블입니다.

from transformers.cache_utils import Cache, DynamicCache, StaticCache  
# 모델 실행 중에 캐시를 효율적으로 관리하기 위한 유틸리티입니다.
# Cache: 일반 캐시 클래스, 
# DynamicCache: 동적 캐시 관리, 
# StaticCache: 정적 캐시 관리

from transformers.modeling_outputs import (
    BaseModelOutputWithPast,  # 기본 모델 출력 구조로, 이전 토큰 히스토리를 포함합니다.
    CausalLMOutputWithPast,  # 언어 모델링 작업에서 이전 토큰 히스토리와 함께 출력되는 구조입니다.
    QuestionAnsweringModelOutput,  # 질문-답변 태스크를 위한 모델 출력 구조입니다.
    SequenceClassifierOutputWithPast,  # 시퀀스 분류 작업에서 이전 히스토리를 포함한 출력 구조입니다.
    TokenClassifierOutput,  # 토큰 분류 작업을 위한 모델 출력 구조입니다.
)

from transformers.modeling_rope_utils import ROPE_INIT_FUNCTIONS  
# 상대적 위치 인코딩(Relative Position Encoding)을 처리하는 함수들입니다.

from transformers.modeling_utils import PreTrainedModel  
# 모든 사전 학습된 모델의 기반이 되는 클래스입니다. 다양한 공통 기능을 제공합니다.

from transformers.pytorch_utils import ALL_LAYERNORM_LAYERS  
# 레이어 정규화에 사용하는 모든 레이어 종류를 포함한 상수로, 레이어 정규화를 처리할 때 사용됩니다.

from transformers.utils import (
    add_start_docstrings,  # 모델 클래스와 함수에 설명을 추가하는 데 사용됩니다.
    add_start_docstrings_to_model_forward,  # 모델의 `forward` 메서드에 설명을 추가하는 데 사용됩니다.
    is_flash_attn_greater_or_equal_2_10,  # 플래시 어텐션의 버전이 2.10 이상인지 확인하는 유틸리티 함수입니다.
    logging,  # 모델 실행 과정에서 로그 출력을 관리하는 데 사용됩니다.
    replace_return_docstrings,  # 리턴 값에 대한 설명을 대체하는 유틸리티입니다.
    )

from configuration_llama import LlamaConfig  
# LLaMA 모델의 설정 클래스로, 모델의 하이퍼파라미터와 구조 설정을 관리합니다.


### 1. **LlamaRMSNorm과 기존 정규화 방식의 차이**

LlamaRMSNorm은 **RMSNorm**(Root Mean Square Normalization)을 기반으로 한 정규화 방식입니다. 기존의 **Layer Normalization (LayerNorm)**과는 다음과 같은 차이점이 있습니다:

- **LayerNorm (층 정규화)**:
  - **동작 원리**: LayerNorm은 각 입력 벡터의 모든 차원에 대해 평균과 분산을 계산하여 정규화합니다. 각 벡터의 평균을 0으로, 분산을 1로 맞추는 것이 목표입니다.
  - **계산 방식**: 각 벡터에서 평균과 분산을 구해 그 값을 빼고 나누는 방식으로 정규화됩니다.
  - **비용**: 전체 벡터에 대해 평균과 분산을 계산해야 하므로 연산 비용이 상대적으로 큽니다.

- **RMSNorm (RMS 정규화)**:
  - **동작 원리**: RMSNorm은 벡터의 평균을 계산하지 않고, **RMS(Root Mean Square)**, 즉 루트 평균 제곱만을 이용해 정규화합니다. 벡터의 각 차원 값에 대해 제곱한 후 평균을 구하고, 이를 통해 정규화를 합니다.
  - **계산 방식**: 입력 벡터의 분산만을 고려하고 평균을 따로 계산하지 않으므로 계산이 더 간단하고 빠릅니다.
  - **차이점**: LayerNorm은 평균과 분산을 계산해 정규화하는 반면, RMSNorm은 분산만을 고려하는 방식입니다. 따라서 RMSNorm은 더 적은 계산 비용을 요구합니다.

**주요 차이점**은 RMSNorm은 벡터의 평균을 무시하고 분산만을 정규화하기 때문에 계산이 더 단순하고 속도가 빠를 수 있다는 점입니다. 특히 대규모 모델에서는 이 차이가 성능에 영향을 줄 수 있습니다.

### 2. **히든 사이즈가 학습 가능한 파라미터로 설정된 이유**

LlamaRMSNorm의 정규화에서 `hidden_size`가 학습 가능한 파라미터로 설정된 이유는 다음과 같습니다:

- **학습 가능한 가중치**: LlamaRMSNorm에서는 `self.weight`라는 학습 가능한 가중치가 사용됩니다. 이는 일반적인 정규화 방법에서 **스케일링(Scaling)**을 담당하는 역할을 합니다. 정규화된 값에 이 학습 가능한 가중치를 곱하는 것은 모델이 학습 중에 데이터를 적절히 스케일링하고 조정할 수 있도록 해줍니다.
  - 이 가중치는 각 차원에 대해 다른 값을 가질 수 있으며, 학습을 통해 최적화됩니다. 정규화된 결과를 그대로 사용하는 것이 아니라, 학습된 가중치를 곱해 데이터의 크기를 조정합니다.
  
- **정규화된 값의 유연성**: 정규화는 입력값의 분산을 조절하지만, 그 자체로는 모델이 각 층에서 출력값의 크기를 최적으로 조절하지 못할 수 있습니다. 이를 해결하기 위해 정규화된 값에 학습 가능한 가중치를 곱해, **정규화 이후의 출력값을 조정**하게 됩니다.
  
- **RMSNorm에서의 학습 가능한 파라미터**: RMSNorm에서 이 가중치는 **정규화된 값의 스케일을 조정**하는데 사용됩니다. 즉, 모델은 각 차원의 가중치를 학습하면서 정규화된 값의 크기를 유연하게 조절할 수 있습니다. 이는 모델이 학습하는 동안 적절한 출력값 크기를 유지하는 데 도움이 됩니다.

### 3. **RMSNorm이 분산만을 고려했는데 정규화가 가능한 이유**

RMSNorm에서 분산만을 고려해도 정규화가 가능한 이유는 분산만을 이용해 **입력 값의 스케일(크기)을 조절**하는 것이 목적이기 때문입니다. LayerNorm은 평균과 분산을 함께 사용해 정규화하지만, RMSNorm은 벡터의 크기만을 제어하는 데 집중하여 평균을 무시하고, **루트 평균 제곱(Root Mean Square, RMS)**를 사용해 정규화합니다.

RMSNorm은 벡터가 얼마나 퍼져 있는지를 나타내는 분산을 통해, 벡터의 크기를 일정하게 유지할 수 있습니다. 즉, 분산만으로도 입력 벡터의 **스케일을 고르게 만들어** 신경망 학습 과정에서 안정적인 출력을 얻을 수 있습니다. 평균을 고려하지 않더라도, 분산만으로도 벡터의 크기를 적절하게 조정할 수 있는 것이죠.

### 4. **RMSNorm의 수식**

RMSNorm의 수식은 다음과 같습니다.

#### 주어진 입력 벡터 \( x \)의 각 차원 \( i \)에 대해:

1. RMS (Root Mean Square)를 계산:
   $$text{RMS}(x) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} x_i^2}$$
   여기서 \( n \)은 벡터의 차원 수, \( x_i \)는 입력 벡터의 \( i \)번째 값입니다.

2. 정규화:
   $$hat{x}_i = \frac{x_i}{\text{RMS}(x)}$$
   이는 벡터의 각 값 \( x_i \)를 RMS로 나누어 크기를 정규화하는 과정입니다.

3. 학습 가능한 가중치 \( g \)를 적용:
   $$y_i = g_i \cdot \hat{x}_i$$
   여기서 \( g \)는 각 차원에 대한 학습 가능한 스케일링 파라미터입니다. 정규화된 값에 가중치를 곱해 모델이 학습 중에 각 차원의 출력을 조정할 수 있습니다.

4. 최종 출력:

   $$y = g \cdot \frac{x}{\text{RMS}(x)}$$

### 5. **연산 과정 예시**

예를 들어, 다음과 같은 3차원 벡터 $x$가 있다고 가정합시다:

$$x = [3, 4, 12]$$

1. **각 차원에 대한 제곱을 계산**:
   
   $$x^2 = [9, 16, 144]$$

2. **제곱한 값의 평균을 계산**:
   
   $$frac{1}{3} \sum x_i^2 = \frac{9 + 16 + 144}{3} = \frac{169}{3} \approx 56.33$$

3. **RMS (Root Mean Square)를 계산**:
   
   $$text{RMS}(x) = \sqrt{56.33} \approx 7.51$$

4. **각 차원의 값 $x_i$를 RMS로 나눠 정규화**:
   
   $$hat{x} = \frac{x}{\text{RMS}(x)} = \left[ \frac{3}{7.51}, \frac{4}{7.51}, \frac{12}{7.51} \right] \approx [0.40, 0.53, 1.60]$$

5. **학습 가능한 가중치 \( g \)를 곱해 최종 출력을 얻음** (예를 들어, 가중치가 \( g = [1.5, 2.0, 0.8] \)이라 가정):
   
   $$y = g \cdot \hat{x} = [1.5 \cdot 0.40, 2.0 \cdot 0.53, 0.8 \cdot 1.60] \approx [0.60, 1.06, 1.28]$$

따라서, 정규화된 후의 최종 출력 벡터  $y$는:

$$y \approx [0.60, 1.06, 1.28]$$

이 과정에서 분산을 통해 RMS를 계산하고, 그 값을 기반으로 벡터의 크기를 정규화한 후 학습 가능한 가중치를 곱해 최종적으로 각 차원의 크기를 조정하는 결과를 얻게 됩니다.

이렇게 RMSNorm은 입력 벡터의 **크기만**을 조정하여 안정적인 학습을 지원하는 정규화 방식입니다.

In [18]:
class LlamaRMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        LlamaRMSNorm은 T5LayerNorm과 동일한 역할을 하는 RMSNorm 레이어입니다.

        파라미터:
        - hidden_size (int): 히든 사이즈, 첫번째 레이어에서는 임베딩 벡터의 차원이고 이후 레이어에서는 이전 레이어의 출력이 됩니다.
        - eps (float): 분산 계산 시 0으로 나누는 것을 방지하기 위한 작은 값입니다.
        """
        super().__init__()
        self.weight = nn.Parameter(
            torch.ones(hidden_size)
        )  # 학습 가능한 가중치 파라미터입니다.
        self.variance_epsilon = eps  # 분산 계산 시 사용되는 작은 값입니다.

    def forward(self, hidden_states):
        """
        입력된 히든 스테이트에 RMS 노름을 적용합니다.

        파라미터:
        - hidden_states (torch.Tensor): 입력 텐서로, 형태는 (배치 크기, 시퀀스 길이, 히든 크기)입니다.

        반환값:
        - torch.Tensor: RMS 노름이 적용된 텐서입니다.
        """
        input_dtype = hidden_states.dtype  # 입력의 데이터 타입을 저장합니다.
        hidden_states = hidden_states.to(
            torch.float32
        )  # float32 타입으로 변환하여 수치적 안정성을 확보하기 위해 연산 중에는 더 높은 정밀도로 계산합니다.
        variance = hidden_states.pow(2).mean(
            -1, keepdim=True
        )  # 마지막 차원에 대한 분산을 계산합니다.
        hidden_states = hidden_states * torch.rsqrt(
            variance + self.variance_epsilon
        )  # 정규화합니다.
        return self.weight * hidden_states.to(
            input_dtype
        )  # 메모리와 성능을 효율화하기 위해 다시 원래의 데이터 형식으로 변환하여 가중치와 곱합니다.

    def extra_repr(self):
        return f"{tuple(self.weight.shape)}, eps={self.variance_epsilon}"  # 레이어의 추가 정보를 문자열로 반환합니다.


# RMSNorm 레이어를 모든 LayerNorm 레이어의 리스트에 추가합니다.
ALL_LAYERNORM_LAYERS.append(LlamaRMSNorm)